## Imports


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import glob
import os
import random
import copy

## Definition of Tensorflow dataflow starts here

Following Variables and placeholders are defined

In [3]:
img_height = 200
img_width = 200
num_channels = 3
num_classes = 5
img_size_flat = 200*200*3

filter_size1 = 11
num_filters1 = 32

filter_size2 = 5
num_filters2 = 32

filter_size3 = 2
num_filters3 = 64
#x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

#x_image = tf.reshape(x, [-1, img_height, img_width, num_channels], name='x_image')

x_image = tf.placeholder(tf.float32, shape=[None, img_height, img_width, num_channels])

y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name= 'y_true')

y_true_cls = tf.argmax(y_true, axis = 1, name='y_true_cls')

weights = {
    'h1': tf.Variable(tf.truncated_normal([filter_size1, filter_size1, num_channels, num_filters1], stddev=0.05)),
    'h2': tf.Variable(tf.truncated_normal([filter_size2, filter_size2, 32, num_filters2], stddev=0.05)),
    'newh1': tf.Variable(tf.truncated_normal([filter_size3, filter_size3, 32, num_filters3], stddev=0.05)),
    'h3': tf.Variable(tf.truncated_normal([7*7*64, 7*7*64],stddev=0.05)),
    'newh2': tf.Variable(tf.truncated_normal([7*7*64, 7*7*64],stddev=0.05)),
    'newh3': tf.Variable(tf.truncated_normal([7*7*64, 1024],stddev=0.05)),
    'out': tf.Variable(tf.truncated_normal([1024, 5],stddev=0.05))

}
biases = {
    'b1': tf.Variable(tf.constant(0.05, shape=[num_filters1])),
    'b2': tf.Variable(tf.constant(0.05, shape=[num_filters2])),
    'newb1': tf.Variable(tf.constant(0.05, shape=[num_filters3])),
    'b3': tf.Variable(tf.constant(0.05, shape=[7*7*64])),
    'newb2': tf.Variable(tf.constant(0.05, shape=[7*7*64])),
    'newb3': tf.Variable(tf.constant(0.05, shape=[1024])),
    'out': tf.Variable(tf.constant(0.05,shape=[5]))

}


### First Convolution Layer

This layer produce a tensor of size (batch_size X 25 X 25 X 32) after a max_pool layer

In [4]:
conv1 = tf.nn.conv2d(input=x_image, filter= weights['h1'], strides=[1,4,4,1], padding='SAME') 
bias_add1 = tf.nn.bias_add(conv1, biases['b1'])
relu1 = tf.nn.relu(bias_add1)
pool1 = tf.nn.max_pool(value=relu1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(conv1)
print(pool1)

Tensor("Conv2D:0", shape=(?, 50, 50, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 25, 25, 32), dtype=float32)


### Second Convolution Layer

This layer produce a tensor of size (batch_size X 13 X 13 X 32) after a max_pool layer

In [5]:
conv2 = tf.nn.conv2d(input=pool1, filter= weights['h2'], strides=[1,1,1,1], padding='SAME') 
bias_add2 = tf.nn.bias_add(conv2, biases['b2'])
pool2 = tf.nn.max_pool(value=bias_add2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
relu2 = tf.nn.relu(pool2)
print(conv2)
print(pool2)

Tensor("Conv2D_1:0", shape=(?, 25, 25, 32), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 13, 13, 32), dtype=float32)


### Third Convolution Layer

This layer produce a tensor of size (batch_size X 7 X 7 X 64) after a max_pool layer

In [6]:
conv3 = tf.nn.conv2d(input=relu2, filter= weights['newh1'], strides=[1,1,1,1], padding='SAME') 
bias_add3 = tf.nn.bias_add(conv3, biases['newb1'])
pool3 = tf.nn.max_pool(value=bias_add3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
relu3 = tf.nn.relu(pool3)
print(conv3)
print(pool3)
print(relu3)

Tensor("Conv2D_2:0", shape=(?, 13, 13, 64), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Relu_2:0", shape=(?, 7, 7, 64), dtype=float32)


### Now this last layer will be flattened so that it could be passed by a Dense Layer

Dense Layer is basically an array of features flattened

In [7]:
pool2_flat = tf.reshape(relu3, [-1, 7*7*64])
pool2_flat

<tf.Tensor 'Reshape:0' shape=(?, 3136) dtype=float32>

## First Dense Layer 

Number of Neurons in this layer = 3136

In [8]:
fc_layer = tf.add(tf.matmul(pool2_flat, weights['h3']), biases['b3'], name='fc_layer')
relu4 = tf.nn.relu(fc_layer)
relu4

<tf.Tensor 'Relu_3:0' shape=(?, 3136) dtype=float32>

## Second Dense Layer 

Number of Neurons in this layer = 3136

In [9]:
fc_layer_1 = tf.add(tf.matmul(relu4, weights['newh2']), biases['newb2'], name='fc_layer_1')
relu5 = tf.nn.relu(fc_layer_1)

## Third Dense Layer 

Number of Neurons in this layer = 1024

In [10]:
fc_layer_l = tf.add(tf.matmul(relu5, weights['newh3']), biases['newb3'], name='fc_layer_l')
relu6 = tf.nn.relu(fc_layer_l)
relu6

<tf.Tensor 'Relu_5:0' shape=(?, 1024) dtype=float32>

## Dropout Layer

Some Neurons are dropped due to drecreasing significance.

0.25 is the probability that a neuron will be dropped

In [11]:
dropout = tf.nn.dropout(relu6, 0.25)
dropout

<tf.Tensor 'dropout/mul:0' shape=(?, 1024) dtype=float32>

## Final Output Layer 

Number of Neurons in this layer = Number of Classes (Outputs) = 5

In [12]:
fc_layer2 = tf.add(tf.matmul(dropout, weights['out']), biases['out'], name='fc_layer2')

Variables for predicted class and probabilites.

In [13]:
y_pred_cls = tf.argmax(fc_layer2, axis=1, name='y_pred_cls')
y_pred_prbl = tf.nn.softmax(fc_layer2, name='y_pred_prbl')

## Definition of Cost Function 

Here Categorical Cross Entropy is used which is widely used to define loss in categorical classification.

In [15]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc_layer2,
                                                        labels=y_true)
cost = tf.reduce_mean(cross_entropy, name='cost')

## Definition of Optimizer

Adam Optimizer is used here which is an optimization of Gradient Descent optimizer with 0.001 as learning rate.

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3, name='optimizer').minimize(cost)

Variables for Accuracy

In [17]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

## Running the Tensorflow dataflow Graph

Tensorflow dataflow graph is now executed in a session.

Images are passed into the Placeholders defined in the dataflow graph in the form of batches of size 100.           
Optimizer is executed for each batch.                                                                                Number of iterations are = 1486

In [18]:
classes = ['Jackets', 'Jeans', 'Skirts', 'Sunglasses', 'Tops']

cwd = os.getcwd()

list_files = []

list_files = os.listdir('/home/shresth/Far/')
for i in range(len(list_files)):
    list_files[i] = '/home/shresth/Far/' + list_files[i]

train_batch_size = 100
number_of_iterations = 15
num_train = 9900

sess = tf.Session()
sess.run(tf.global_variables_initializer())
itt = 1

for _ in range(number_of_iterations):
    ii = 0
    random.shuffle(list_files)
    while ii < num_train:
        jj = min(ii + train_batch_size, num_train)

        y_true_batch = []

        names = list_files[ii:jj]   
        for nm in names:    
            for cl in classes:
                if cl in nm:
                    one_hot = [0., 0., 0., 0., 0.]
                    one_hot[classes.index(cl)] = 1.
                    y_true_batch.append(one_hot)
        y_true_batch = np.array(y_true_batch)

        x_batch = [cv2.imread(nm) for nm in names]
        x_batch = np.array(x_batch)

        print(x_batch.shape)
        print(y_true_batch.shape)

        sess.run(optimizer, feed_dict={x_image: x_batch, y_true: y_true_batch})

        acc = sess.run(accuracy, feed_dict={x_image: x_batch, y_true: y_true_batch})

        print('Optimization iteration = ',itt+1, 'Accuracy = ',acc*100,'%')

        print("Hi")

        ii = jj
        itt+=1

(100, 200, 200, 3)
(100, 5)
Optimization iteration =  2 Accuracy =  30.000001192092896 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  3 Accuracy =  23.999999463558197 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  4 Accuracy =  28.999999165534973 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  5 Accuracy =  41.999998688697815 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  6 Accuracy =  33.000001311302185 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  7 Accuracy =  41.999998688697815 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  8 Accuracy =  33.000001311302185 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  9 Accuracy =  40.00000059604645 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  10 Accuracy =  36.000001430511475 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  11 Accuracy =  30.000001192092896 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  12 Accuracy =  33.0000013

Optimization iteration =  91 Accuracy =  83.99999737739563 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  92 Accuracy =  82.99999833106995 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  93 Accuracy =  88.99999856948853 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  94 Accuracy =  80.0000011920929 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  95 Accuracy =  81.99999928474426 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  96 Accuracy =  81.00000023841858 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  97 Accuracy =  81.99999928474426 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  98 Accuracy =  81.00000023841858 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  99 Accuracy =  81.99999928474426 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  100 Accuracy =  79.00000214576721 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  101 Accuracy =  80.0000011920929 %
Hi
(100, 200, 200, 

(100, 200, 200, 3)
(100, 5)
Optimization iteration =  180 Accuracy =  85.00000238418579 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  181 Accuracy =  83.99999737739563 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  182 Accuracy =  87.99999952316284 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  183 Accuracy =  81.99999928474426 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  184 Accuracy =  85.00000238418579 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  185 Accuracy =  81.99999928474426 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  186 Accuracy =  83.99999737739563 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  187 Accuracy =  83.99999737739563 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  188 Accuracy =  82.99999833106995 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  189 Accuracy =  89.99999761581421 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  190 Accuracy =  

Optimization iteration =  268 Accuracy =  97.00000286102295 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  269 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  270 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  271 Accuracy =  89.99999761581421 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  272 Accuracy =  92.00000166893005 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  273 Accuracy =  93.99999976158142 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  274 Accuracy =  93.00000071525574 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  275 Accuracy =  93.99999976158142 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  276 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  277 Accuracy =  93.00000071525574 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  278 Accuracy =  95.99999785423279 %
Hi
(100,

Optimization iteration =  357 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  358 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  359 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  360 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  361 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  362 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  363 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  364 Accuracy =  97.00000286102295 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  365 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  366 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  367 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3

...

Optimization iteration =  1080 Accuracy =  97.00000286102295 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1081 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1082 Accuracy =  94.9999988079071 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1083 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1084 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1085 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1086 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1087 Accuracy =  97.00000286102295 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1088 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1089 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1090 Accuracy =  99.00000095367432 %
Hi
(100, 2

Optimization iteration =  1171 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1172 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1173 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1174 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1175 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1176 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1177 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1178 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1179 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1180 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1181 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  

Optimization iteration =  1262 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1263 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1264 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1265 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1266 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1267 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1268 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1269 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1270 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1271 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1272 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1273 Accuracy =  99.0000

Optimization iteration =  1356 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1357 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1358 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1359 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1360 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1361 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1362 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1363 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1364 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1365 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1366 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1367 Accurac

Optimization iteration =  1447 Accuracy =  91.00000262260437 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1448 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1449 Accuracy =  97.00000286102295 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1450 Accuracy =  93.99999976158142 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1451 Accuracy =  95.99999785423279 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1452 Accuracy =  99.00000095367432 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1453 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1454 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1455 Accuracy =  100.0 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1456 Accuracy =  97.00000286102295 %
Hi
(100, 200, 200, 3)
(100, 5)
Optimization iteration =  1457 Accuracy =  98.00000190734863 %
Hi
(100, 200, 200, 3)
(100, 5)
Op

In [20]:
model_path = "Jack Class new/JC"
saver = tf.train.Saver()
save_path = saver.save(sess, model_path)

print('Model Saved in file : ',save_path)

Model Saved in file :  Jack Class new/JC


Model is now saved in the destination folder.                                                                       It will contain the weights for the Variables- "weights" and "biases"

In [26]:
sess.close()

array([[ 6.2078876 , -5.13688   , -0.7940725 , -5.8865995 ,  0.67557275]],
      dtype=float32)

## Closing the Session at last